In [11]:
# import requests
# import os
import random
import json
# import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier

In [12]:
# Загружаем файл с диалогами для обучения модели
filename = "intents_dataset.json"
# Считываем файл в словарь
with open(filename, 'r', encoding='UTF-8') as file:
    data = json.load(file)   
# Создаем массивы фраз и интентов для обучения
X = []
y = []
for name in data:
    for phrase in data[name]['examples']: 
        X.append(phrase) # добавляем фразы в вектор X из массива examples
        y.append(name) # к каждой фразе examples добавляем соответсвующий ключ Y.(первое вхождение)  
    for phrase in data[name]['responses']:
        X.append(phrase) # добавляем фразы в вектор X из массива responses
        y.append(name) #к каждой фразе responses добавляем соответсвующий ключ Y.(первое вхождение)

# Векторизуем наши фразы X
vectorizer = CountVectorizer() # создаём объект
vectorizer.fit(X) #обучаем векторайзер по нашему X (создаём список уникальных слов)
X_vec = vectorizer.transform(X) # получаем численное выражение наших текстов 

In [13]:
# Создаём модель
model = MLPClassifier()
# Обучаем модель на численных выражениях наших текстах и массиве правильных ответов Y
model.fit(X_vec, y)

c:\anaconda\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier()

In [14]:
model.score(X_vec, y) # посчитаем accuracy нашей модели

0.8653204565408252

In [15]:
class bot_facts:
  def get_intent(text): # метод который принимает текст пользователя, векторизует и возвращает "у" - какая тема разговора
    # сначала преобразуем текст пользователя в числа
    text_vec = vectorizer.transform([text])
    # берем элемент номер 0 - для того(так как predikt возвращает список с одним элементом), чтобы избавиться от формата "список", который необходим для векторизации и машинного обучения
    return model.predict(text_vec)[0]
  def get_response(intent): # метод в зависимости от темы разговора, возвращает ответ пользователю, случайным образом.
    return random.choice(data[intent]['responses'])
  def bot(text): # метод который получает текст пользователя и возвращает ответ
    intent = bot_facts.get_intent(text)
    answer = bot_facts.get_response(intent)
    return answer

In [16]:
# функции запускаются асинхронно(количество одновременнх пользователей не ограниченно)
import nest_asyncio 
nest_asyncio.apply()

In [17]:
# токен полученный у BotFather
TOKEN = '5663527788:AAF9w39f6XWKodCNcnRKmof6DHdF1he9SYo'

In [18]:
#  Update - информация, полученная с сервера (новые сообщения, новые контакты)
#  С сервера регулярно регулярно приходят Update'ы с новой информацией
from telegram import Update 
from telegram.ext import ApplicationBuilder #  Инструмент чтобы создавать и настраивать приложение (телеграм бот)
from telegram.ext import MessageHandler #  Handler (обработчик) - создать реакцию (функцию) на действие
from telegram.ext import filters #на какие события будет происходить реакция

In [19]:
# Асинхронная функция для MessageHandler'а, вызывать ее при каждом сообщении боту
async def reply(update: Update, context) -> None:
    user_text = update.message.text # создаётся переменная и присваивается ей сообщение пользователя
    reply = bot_facts.bot(user_text) # создаётся переменная и присваивается ей ответ бота
    print('<', user_text)
    print('>', reply)

    await update.message.reply_text(reply) # передаём ответ бота конкретному пользователю

# Создаем объект приложения - связываем его с токеном
app = ApplicationBuilder().token(TOKEN).build()
# Создаем обработчик текстовых сообщений на тип события "текст" и возвращает функцию 'reply'
handler = MessageHandler(filters.Text(), reply)
# Добавляем обработчик в приложение
app.add_handler(handler)
# Запускаем приложение
app.run_polling()

< привет
> Привет,цель Вашего Визита
< какой сегодня день?
> Взгляните на календарь
< что делаешь
> анекдоты травлю, хочешь?
< расскажи анекдот
> калькулятор - это устаревший робот,ахахахах.
< ты робот?
> в наше время все границы и определения размываются..
< /start
> Здравстуйте
< /start
> Трям!
< Привет
> Привет, как прошёл день
< Средне
> Здарова
< хай
> Драсьте. Как вас зовут?
< Олег
> Ты прикалываешься
< Нет
> Нет
< рлоро
> день добрый
< что скажешь
> Тра-ля-ля
< поговори со мной
> Могу рассказать про космос, природу и как делают вишневые пирожки в МакДональдсе
< /start
> И тебе привет
< привет!
> Доброго времени суток!
< что делать?
> Послушайте музыку
< не хочу
> Сколько программистов надо, чтобы вкрутить одну лампочку? – Ни одного. В этом случае отсутствие света – проблема на стороне железа.
< /start
> Я немного занят
< /start
> Салютик =D
< не смешно
> Соври хоть раз, скажи, что я твой друг!
< нет
> Нет
< /start
> Алоха
< Привет
> Эй, ты как?
< что поесть?
> Отличное занятие! 